**AI agent-based Deep Research**

In [29]:
import os

#  Tavily API key
os.environ["TAVILY_API_KEY"] = "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"


In [30]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# Load the FLAN-T5 model
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=512)
llm = HuggingFacePipeline(pipeline=hf_pipeline)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-30-7459f6cd2127>:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [31]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Prompt to generate an answer from research data
prompt = PromptTemplate.from_template("""
You are a research expert. Based on the following collected information,
write a clear and concise answer:

{data}
""")

answer_chain = LLMChain(llm=llm, prompt=prompt)

def answer_drafting_agent(data):
    return answer_chain.run(data=data)


In [41]:
from langchain.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()

def research_agent(query):
    results = search_tool.run(query)
    return "\n\n".join([r['content'] for r in results])


In [42]:
from langgraph.graph import StateGraph, END
from langchain.schema.runnable import RunnableLambda
from typing import TypedDict, Annotated

# Define state schema (structure of data passed between nodes)
class AgentState(TypedDict):
    query: str
    data: str
    answer: str

# Research step
def research_step(state: AgentState) -> dict:
    query = state["query"]
    data = research_agent(query)
    return {"data": data}

# Drafting step
def draft_step(state: AgentState) -> dict:
    data = state["data"]
    answer = answer_drafting_agent(data)
    return {"answer": answer}

# Build LangGraph with schema
builder = StateGraph(AgentState)

builder.add_node("research", RunnableLambda(research_step))
builder.add_node("draft", RunnableLambda(draft_step))

builder.set_entry_point("research")
builder.add_edge("research", "draft")
builder.add_edge("draft", END)

# Compile the graph
compiled_graph = builder.compile()


In [44]:
query = "tell me what is AI"

# Run the dual-agent system
result = compiled_graph.invoke({"query": query})

# ✅ Fix: result is a dict, not a list — so we can access keys safely
print("✅ Final Answer:\n")
print(result.get("answer", "❌ No answer found in result."))


✅ Final Answer:

Artificial intelligence (AI) is the theory and development of computer systems capable of performing tasks that historically required human intelligence, such as reasoning, making decisions, and identifying patterns.
